pip install xgboost

In [95]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import joblib

In [96]:
df = pd.read_csv("AirQualityUCI.csv", delimiter=';', decimal=',', parse_dates=[[0, 1]], na_values=-200)

C:\Users\Sugam\AppData\Local\Temp\ipykernel_46312\3248867744.py:1: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv("AirQualityUCI.csv", delimiter=';', decimal=',', parse_dates=[[0, 1]], na_values=-200)
C:\Users\Sugam\AppData\Local\Temp\ipykernel_46312\3248867744.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("AirQualityUCI.csv", delimiter=';', decimal=',', parse_dates=[[0, 1]], na_values=-200)


In [97]:
df.head()

,Date_Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004 18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,NaN,NaN
1,10/03/2004 19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN
2,10/03/2004 20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN
3,10/03/2004 21.00.00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,NaN,NaN
4,10/03/2004 22.00.00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,NaN,NaN


In [98]:
# Drop unnecessary columns and handle missing values
df.drop(['Unnamed: 15', 'Unnamed: 16'], axis=1, inplace=True)
df.dropna(inplace=True)

# Convert Date_Time column to datetime format
df['Date_Time'] = pd.to_datetime(df['Date_Time'], format='%d/%m/%Y %H.%M.%S')

In [99]:
df = df.set_index('Date_Time').sort_index()

In [100]:
# Feature engineering
def create_features(df, target='CO(GT)', lags=3):
    df = df.copy()
    
    # Temporal features
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    
    # Lag features
    for i in range(1, lags+1):
        df[f'{target}_lag_{i}'] = df[target].shift(i)
    
    # Rolling statistics
    df[f'{target}_roll_mean_24h'] = df[target].rolling(24).mean()
    df[f'{target}_roll_std_24h'] = df[target].rolling(24).std()
    
    return df.dropna()

df_engineered = create_features(df)

In [101]:
# Train-test split (chronological)
X = df_engineered.drop('CO(GT)', axis=1)
y = df_engineered['CO(GT)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [102]:
#Model training and evaluation
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42)
}

results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    metrics = {
        'Model': name,
        'MAE': mean_absolute_error(y_test, preds),
        'RMSE': np.sqrt(mean_squared_error(y_test, preds))
    }
    results.append(metrics)
    
    # Save models
    joblib.dump(model, f'{name.replace(" ", "_").lower()}.joblib')

# Display results
print(pd.DataFrame(results))

               Model       MAE      RMSE
0  Linear Regression  0.182926  0.235589
1      Random Forest  0.239186  0.314810
2            XGBoost  0.191925  0.254382
